In [1]:
from nariflow import start_initializer

start_initializer().initializer('tape')

In [2]:
from nariflow import Variable
from nariflow import optimizer
from nariflow import GradientTape
#from nariflow import calc_gradient
from nariflow import layer
from nariflow.models import Model
from nariflow import functions as f
from nariflow.core import elementary_function as ef
from nariflow.core import shape_function as sf
from nariflow.utils import DataSet
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from tensorflow.keras.layers.experimental import preprocessing

In [3]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

1679360/1668792 [==============================] - 0s 0us/step


In [4]:
# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop un-used columns.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

In [5]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

7383 train examples
1846 validation examples
2308 test examples


In [6]:
train_processing = pd.concat([pd.get_dummies(train.loc[:, train.apply(lambda x : x.dtype == 'object')]),
           train.loc[:,train.apply(lambda x : x.dtype != 'object')].apply(lambda x : (x - min(x)) / (max(x) - min(x)) )], 
          axis = 1)

In [7]:
X_train = train_processing.drop(['target'], axis = 1).to_numpy()

In [8]:
test_processing = pd.concat([pd.get_dummies(test.loc[:, test.apply(lambda x : x.dtype == 'object')]),
           test.loc[:,test.apply(lambda x : x.dtype != 'object')].apply(lambda x : (x - min(x)) / (max(x) - min(x)) )], 
          axis = 1)

test_processing = pd.concat([pd.DataFrame(columns = train_processing.columns),
           test_processing])[train_processing.columns].fillna(0)

In [9]:
y_test = np.array(pd.get_dummies(test_processing.loc[:,'target']))

In [10]:
X_test= test_processing.drop(['target'], axis = 1).to_numpy()

In [11]:
y_train = np.array(pd.get_dummies(train_processing.loc[:,'target']))

In [12]:
lr = 0.2
max_iter = 10000
hidden_size = 200
out_size = 2

In [13]:
class Models(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        self.l1 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l2 = layer.Linear(hidden_size, initializer_func='he_uniform')
        self.l3 = layer.Linear(out_size, initializer_func='he_uniform')
        
    def forward(self, x):
        y = self.l1(x)
        y = f.activation.relu(y)
        y = self.l2(y)
        y = f.activation.relu(y)
        y = self.l3(y)
        y = f.activation.softmax(y)
        return y

In [14]:
lr = 0.001

In [15]:
model = Models(hidden_size, out_size)

optimizers = optimizer.Adam(lr)

optimizers.setup(model)

In [16]:
batch_size = 50

In [17]:
X_train = Variable(np.array(X_train))

y_train = Variable(y_train)

dataset = DataSet(X_train, y_train)

dataset.batch_setup(batch_size)

In [18]:
def train_step(X_train, y_train):
    with GradientTape() as tape:
        
        y_pred = model(X_train)
        loss = f.loss.categorical_crossentropy(y_train, y_pred)
        
    tape.CalcGradient()
    optimizers.update()
    
    if j % 5 == 0:
        print(loss.data)

In [19]:
start_time = time.time()

i = 0

while True:
    for j in range(np.ceil((len(dataset) / batch_size)).astype('int')):

        train_set, val_set = next(dataset)

        train_step(train_set, val_set)

    print('total_time : ', time.time() - start_time)

    dataset.reset()
    
    i += 1
    
    print(i)
    
    if i > 2:
        break

0.6931710143988028
0.6747361489132182
0.6104166374589173
0.6161155502435084
0.6229973030010332
0.57507054591286
0.5603498083641721
0.6208305135455268
0.5732304415842362
0.5585950801140338
0.5835798382805981
0.6019765777592596
0.6155254069849577
0.6075535375960525
0.5889668796566376
0.5811839938514151
0.5622672931382656
0.5450015809450296
0.49629742647393194
0.5649209534835762
0.6165470683459991
0.5469434895707134
0.5394717551673263
0.6194448458927704
0.6135445751981775
0.6253503949262934
0.558652501174907
0.5295177105069991
0.5756519495114181
0.604016578100544
total_time :  0.7114534378051758
1
0.6066421915783191
0.6021742609670013
0.6037662687391486
0.6045256818364615
0.601890679630805
0.5551114018245555
0.535898949619023
0.5831050645946799
0.5380806981871039
0.5391701292160008
0.5685245364308292
0.5079758869372016
0.564234953355271
0.6008969188160939
0.573142291706604
0.5651949059555338
0.535122576481444
0.5349226835119844
0.5027330334931708
0.5426270008911014
0.6101354983362298
0.53

In [20]:
X_test = Variable(np.array(X_test))

In [21]:
y_pred = model(X_test)

In [22]:
f1_score(np.argmax(y_pred.data, axis = -1), 
               np.argmax(y_test.data, axis = -1))

0.8476821192052981

In [23]:
accuracy_score(np.argmax(y_pred.data, axis = -1), 
               np.argmax(y_test.data, axis = -1))

0.7608318890814558

In [25]:
confusion_matrix(np.argmax(y_pred.data, axis = -1),
               np.argmax(y_test.data, axis = -1))

array([[ 220,  177],
       [ 375, 1536]], dtype=int64)

### 텐서플로우 공식 도큐의 Accuracy는 74%이다.
### 결론 : 다중분류도 문제없이 작동한다.